In [1]:
import pandas as pd
import numpy as np

scale = 0.25

# Domain
x = 200*scale
z = 100*scale
y_air = 20*scale
y_moon = 100*scale
y = y_moon + y_air

# Reglith Layer
y_regolith = 4.5*scale

# Crust Layer
y_crust = y_moon - y_regolith

# Frequency and Wavelength
f_max = 2*50e6 # Maximum Relevant Frequency for the Ricker Wavelet
c = 3e8

# Permittivity and Density
density = 2 # g/cm^3
eps_bg = 1.919**density # Background Permittivity for Moon
vel_bg = c / eps_bg**0.5

lambda_0 = c / (f_max * eps_bg**0.5)

# # Fresnel Zone - Radius
# R = 0.5 * (lambda_0 * d)**0.5
# y = 4 * R

# Grid Descretization
dx = np.round(lambda_0 / 12, 3)
dy = np.round(lambda_0 / 12, 3)
dz = np.round(lambda_0 / 12, 3)

nx = int(x / dx)
ny = int(y / dy)
nz = int(z / dz)
print(f"nx: {nx}, ny: {ny}, nz: {nz}")

# x_grid, y_grid, z_grid = np.meshgrid(x_vals, y_vals, z_vals, indexing='xy')

# Time Window
time_window = 2 * y_moon / vel_bg

# Print
print(f"Domain: {x} x {y} x {z}")
print(f"Base Permittivity of Moon: {eps_bg}")
print(f"Frequency: {f_max/2}")
print(f"Wavelength: {lambda_0}")
# print(f"Fresnel Zone Radius: {R}")
print(f"Grid: {dx} x {dy} x {dz}")
print(f"Time Window: {time_window}")


nx: 384, ny: 230, nz: 192
Domain: 50.0 x 30.0 x 25.0
Base Permittivity of Moon: 3.682561
Frequency: 50000000.0
Wavelength: 1.5633142261594581
Grid: 0.13 x 0.13 x 0.13
Time Window: 3.198333333333333e-07


In [2]:
# Editing the Lava Tube Points to Comply with the Domain

data_all = pd.read_csv('../LavaTubeData/LiDAR_InclineCave_TUBE_TLS_points.txt', sep=' ', header=None)
# Convert to numpy arrays X, Y, Z
data_points = data_all.to_numpy()
X = (data_points[:, 0])*scale
Y = (data_points[:, 2])*scale
Z = (data_points[:, 1])*scale

# Find the minimum and maximum values of X, Y, Z, and find their index

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

# Shift all points in X, Y, Z to the center of the domain
X = X - X_min
Y = Y - Y_min
Z = Z - Z_min

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

X = X - (X_min + X_max)/2 + x/2
Y = Y - Y_max + y_moon
Z = Z - (Z_min + Z_max)/2 + z/2

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

print(f"X_min: {X_min}")
print(f"X_max: {X_max}")
print(f"Y_min: {Y_min}")
print(f"Y_max: {Y_max}")
print(f"Z_min: {Z_min}")
print(f"Z_max: {Z_max}")

data_all_shifted = pd.DataFrame(np.column_stack((X, Y, Z)))
data_all_shifted = data_all_shifted.to_numpy()

# Save the new points to vtk
from pyevtk.hl import pointsToVTK
pointsToVTK("LiDAR_InclineCave_TUBE_adjusted", X, Y, Z)

X_min: 8.756347625007038
X_max: 41.24365237499296
Y_min: 18.63000500000004
Y_max: 25.0
Z_min: 4.682617124984972
Z_max: 20.317382875015028


'f:\\Projects\\LunarLeaper\\moon_src\\src\\LiDAR_InclineCave_TUBE_adjusted.vtu'

In [3]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

permittivity_values = 7.6* np.ones_like(X)

In [4]:
#Grid
# x_val = np.linspace(0, x, 5)
# z_val = np.linspace(0, z, 5)
# y_val = np.linspace(0, y_moon, 5)
x_val = np.linspace(0, x, int(nx*scale))
z_val = np.linspace(0, z, int(ny*scale))
y_val = np.linspace(0, y_moon, int(nz*scale))

In [5]:
meshgrid_x, meshgrid_y, meshgrid_z = np.meshgrid(x_val, y_val, z_val, indexing='ij')
points = np.vstack((meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())).T

# Create Delaunay triangulation
tri = Delaunay(np.column_stack((X, Y, Z)))

In [6]:
pointsToVTK("3_Box", meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())

'f:\\Projects\\LunarLeaper\\moon_src\\src\\3_Box.vtu'

In [7]:
# Check which points are within the convex hull
# Check for interpolation
simplex_indices = tri.find_simplex(points)
mask = simplex_indices >= 0

In [8]:
# x, y, z for just the mask
x_mask = meshgrid_x.flatten()[mask]
y_mask = meshgrid_y.flatten()[mask]
z_mask = meshgrid_z.flatten()[mask]
points_cave_edge = np.vstack((x_mask, y_mask, z_mask)).T
print(f"Number of Points in Convex Hull: {len(z_mask)}")
print(f"Number of Points in Total: {len(points_cave_edge)}")

Number of Points in Convex Hull: 3997
Number of Points in Total: 3997


In [16]:
with open('3_moon_basic_filled_tube.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#material: {eps_bg} 1e-10 1 0 moon_lava_base\n")
    f.write(f"#box: 0 0 0 {x} {y_moon} {z} moon_lava_base\n")

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_base n\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 3_moon_basic_filled_tube n \n")

# Output success message
print("gprMax input file '3_moon_basic_filled_tube.in' generated.")

gprMax input file '3_moon_basic_filled_tube.in' generated.


In [9]:
with open('3_moon_basic_filled_tube_v2.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    # receiver_positions = [[0.2, 0.1, 0.0], [0.3, 0.1, 0.0], [0.4, 0.1, 0.0]]
    # for rx in receiver_positions:
    #     f.write(f"#rx: {rx[0]} {rx[1]} {rx[2]}\n")

    f.write(f"#material: {eps_bg} 1e-10 1 0 moon_lava_base\n")
    f.write(f"#box: 0 0 0 {x} {y_moon} {z} moon_lava_base\n")

    eps_r_value = 7.6
    sigma_value = 1e-10
    mu_r_value = 1.0
    sigma_m_value = 0.0
    f.write(f"#material: {eps_r_value} {sigma_value} {mu_r_value} {sigma_m_value} moon_lava_tube \n")
    # Dielectric smoothing needs to be turned off, add higher discretization for numerical stability

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_tube n\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 3_moon_basic_filled_tube_v2 n \n")

# Output success message
print("gprMax input file 'src/3_moon_basic_filled_tube_v2.in' generated.")

gprMax input file 'src/3_moon_basic_filled_tube_v2.in' generated.


In [10]:
from scipy.spatial import ConvexHull

# Calculate the centroid of the original points
centroid_cave = np.mean(points_cave_edge, axis=0)

# Function to shrink each point towards the centroid
def shrink_point(point, centroid, dx=2*dx, dy=2*dy, dz=2*dz):
    direction = point - centroid
    factor = np.array([dx, dy, dz]) / np.linalg.norm(direction)
    shrink_vector = direction * factor
    return point - shrink_vector

# Apply shrinking to all points
points_cave_vacuum_all = np.array([shrink_point(p, centroid_cave, dx, dy, dz) for p in points_cave_edge])

# Compute the new convex hull from shrunken points
convex_cave_vacuum = ConvexHull(points_cave_vacuum_all)

# Extract the new set of coordinates defining the new convex hull
points_cave_vacuum = points_cave_vacuum_all[convex_cave_vacuum.vertices]
print(f"Number of Points in Vacuum Convex Hull: {len(points_cave_vacuum)}")

Number of Points in Vacuum Convex Hull: 184


In [11]:
# pointsToVTK("Vacuum", points_cave_vacuum[:, 0], points_cave_vacuum[:, 1], points_cave_vacuum[:, 2])

In [12]:
with open('4_moon_basic_vacuum_tube.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    # receiver_positions = [[0.2, 0.1, 0.0], [0.3, 0.1, 0.0], [0.4, 0.1, 0.0]]
    # for rx in receiver_positions:
    #     f.write(f"#rx: {rx[0]} {rx[1]} {rx[2]}\n")

    # Define a fractal box and then only create lava tube with corresponding coordinates
    # Then define vaccum with the help of a smaller conex hull

    # 2012. Regolith match
    f.write(f"#material: {eps_bg} 1e-10 1 0 moon_lava_base\n")
    f.write(f"#box: 0 0 0 {x} {y_moon} {z} moon_lava_base\n")

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_base n\n")

    for i in range(len(points_cave_vacuum_all)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {points_cave_vacuum_all[i][0]} {points_cave_vacuum_all[i][1]} {points_cave_vacuum_all[i][2]} {points_cave_vacuum_all[i][0]+dx} {points_cave_vacuum_all[i][1]+dy} {points_cave_vacuum_all[i][2]+dz} free_space n\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 4_moon_basic_vacuum_tube n \n")

# Output success message
print("gprMax input file '4_moon_basic_vacuum_tube.in' generated.")

gprMax input file '4_moon_basic_vacuum_tube.in' generated.


In [13]:
with open('4_moon_basic_vacuum_tube_v2.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#material: {eps_bg} 1e-10 1 0 moon_lava_base\n")
    f.write(f"#box: 0 0 0 {x} {y_moon} {z} moon_lava_base\n")

    eps_r_value = 7.6
    sigma_value = 1e-10
    mu_r_value = 1.0
    sigma_m_value = 0.0
    f.write(f"#material: {eps_r_value} {sigma_value} {mu_r_value} {sigma_m_value} moon_lava_tube \n")

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_tube n\n")

    for i in range(len(points_cave_vacuum_all)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {points_cave_vacuum_all[i][0]} {points_cave_vacuum_all[i][1]} {points_cave_vacuum_all[i][2]} {points_cave_vacuum_all[i][0]+dx} {points_cave_vacuum_all[i][1]+dy} {points_cave_vacuum_all[i][2]+dz} free_space n\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 4_moon_basic_vacuum_tube_v2 n \n")

# Output success message
print("gprMax input file '4_moon_basic_vacuum_tube_v2.in' generated.")

gprMax input file '4_moon_basic_vacuum_tube_v2.in' generated.


In [14]:
with open('7_moon_true_heterogeneous_tube_filled.in', 'w') as f:
# Adjust y in case weak response.
    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#soil_gen_bruggeman_moon: 13 17.8 2.2 3.2 13.5 15.5 7.5 12.5 40.5 47.5 8.5 12.5 soil_reg\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 2.5 0.75 0.75 1.5 1 soil_reg moon_regolith\n")

    f.write(f"#soil_gen_bruggeman_moon: 19 22 1 6 6 10.5 6 11.5 42 47 9.5 10.5 rocks_crust\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 2.5 0.75 0.75 1.5 1 rocks_crust moon_crust\n")

    eps_r_value = 7.6
    sigma_value = 1e-10
    mu_r_value = 1.0
    sigma_m_value = 0.0
    f.write(f"#material: {eps_r_value} {sigma_value} {mu_r_value} {sigma_m_value} moon_lava_tube \n")
    # Dielectric smoothing needs to be turned off, add higher discretization for numerical stability

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_tube n\n")

    f.write(f"#cylinder: {x/2 - 5} {y/2-10} {x/2+5} {y/2-10} {y/2-10} {z/2+5} 5 pec\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 7_moon_true_heterogeneous_tube_filled n \n")

# Output success message
print("gprMax input file '7_moon_true_heterogeneous_tube_filled.in' generated.")

gprMax input file '7_moon_true_heterogeneous_tube_filled.in' generated.


In [15]:
with open('8_moon_true_heterogeneous_tube_vacuum.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#soil_gen_bruggeman_moon: 13 17.8 2.2 3.2 13.5 15.5 7.5 12.5 40.5 47.5 8.5 12.5 soil_reg\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 2.5 0.75 0.75 1.5 1 soil_reg moon_regolith\n")

    f.write(f"#soil_gen_bruggeman_moon: 19 22 1 6 6 10.5 6 11.5 42 47 9.5 10.5 rocks_crust\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 2.5 0.75 0.75 1.5 1 rocks_crust moon_crust\n")

    eps_r_value = 7.6
    sigma_value = 1e-10
    mu_r_value = 1.0
    sigma_m_value = 0.0
    f.write(f"#material: {eps_r_value} {sigma_value} {mu_r_value} {sigma_m_value} moon_lava_tube \n")
    # Dielectric smoothing needs to be turned off, add higher discretization for numerical stability

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_tube n\n")

    for i in range(len(points_cave_vacuum_all)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {points_cave_vacuum_all[i][0]} {points_cave_vacuum_all[i][1]} {points_cave_vacuum_all[i][2]} {points_cave_vacuum_all[i][0]+dx} {points_cave_vacuum_all[i][1]+dy} {points_cave_vacuum_all[i][2]+dz} free_space n\n")

    f.write(f"#cylinder: {x/2 - 5} {y/2-10} {x/2+5} {y/2-10} {y/2-10} {z/2+5} 5 pec\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 8_moon_true_heterogeneous_tube_vacuum n \n")

# Output success message
print("gprMax input file '8_moon_true_heterogeneous_tube_vacuum.in' generated.")

gprMax input file '8_moon_true_heterogeneous_tube_vacuum.in' generated.
